1. Chuẩn bị dữ liệu và nhãn

Tải các package

In [40]:
pip install pandas numpy tensorflow scikit-learn

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [41]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing import image
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

Đọc file CSV chứa thông tin sản phẩm

In [42]:
df = pd.read_csv('./new_image_product.csv')

In [43]:
df.head()

,productid,productname,imagepath
0,6116fdf3-473f-4324-af54-1b950076ef52,Xoài Cát,productImage/6116fdf3-473f-4324-af54-1b950076e...
1,b7237f07-c2bf-48d7-a630-3faaac02cfcc,Gạo ST25,productImage/b7237f07-c2bf-48d7-a630-3faaac02c...
2,95827b65-ec7b-405b-bd26-6558d04211cf,Gạo nếp,productImage/95827b65-ec7b-405b-bd26-6558d0421...
3,aaa373ef-1cfe-4e6a-9c87-eeba7eaec713,Mãng Cầu Xiêm,productImage/aaa373ef-1cfe-4e6a-9c87-eeba7eaec...
4,b11d5edc-9b25-4eab-8f56-aae3476a7082,Rau cải ngồng,productImage/b11d5edc-9b25-4eab-8f56-aae3476a7...


In [44]:
image_dir = ''

Hàm tiền xử lý hình ảnh: resize và chuẩn hóa

In [50]:
def load_and_preprocess_image(image_path, target_size=(224, 224)):
    img = load_img(image_path, target_size=target_size)
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Thêm 1 chiều batch size
    img_array = img_array / 255.0  # Chuẩn hóa ảnh về [0,1]
    return img_array

Chuyển nhãn (productname) thành số

In [46]:
label_encoder = LabelEncoder()
df['encoded_labels'] = label_encoder.fit_transform(df['productname'])

Chuẩn bị dữ liệu cho mô hình

In [51]:
X_data = []
y_data = []

Lặp qua từng dòng trong file CSV

In [58]:

for index, row in df.iterrows():
    image_path = os.path.join(image_dir, row['imagepath'])
    img_array = load_and_preprocess_image(image_path)
    X_data.append(img_array)
    y_data.append(row['productname'])

Chuyển X_data và y_data thành numpy arrays

In [59]:
X_data = np.vstack(X_data)  # Kết hợp tất cả các ảnh thành một mảng duy nhất
y_data = np.array(y_data)

In [60]:
X_data.shape, y_data.shape

((151, 224, 224, 3), (151,))

Chia dữ liệu thành tập huấn luyện và tập kiểm tra

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)

2. Huấn luyện mô hình CNN

In [39]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.optimizers import Adam

# Encode nhãn sản phẩm
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Lưu nhãn đã mã hóa
np.save('label_classes.npy', label_encoder.classes_)

# Tải mô hình ResNet50 pre-trained, không bao gồm các lớp đầu ra
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze các lớp của ResNet50
for layer in base_model.layers:
    layer.trainable = False

# Tạo mô hình mới và thêm các lớp tùy chỉnh
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

# Compile mô hình
model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Huấn luyện mô hình
model.fit(X, y_encoded, epochs=10, batch_size=32, validation_split=0.2)

# Lưu mô hình
model.save('image_search_cnn.h5')


NameError: name 'y' is not defined

 Lưu mô hình và nhãn mã hóa

In [37]:
# Lưu mô hình đã huấn luyện
model.save('image_search_cnn.h5')

# Lưu các nhãn đã mã hóa (label_encoder.classes_)
np.save('label_classes.npy', label_encoder.classes_)
